In [2]:
!pip install cassandra-driver


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import sys
import pandas as pd
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

In [2]:
cloud_config= {
  'secure_connect_bundle': 'C:/Users/SWAMY/Desktop/Cassandra/secure-connect-bigdata.zip'
}

with open("C:/Users/SWAMY/Desktop/Cassandra/bigdata-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()
if session:
    print("connected")
else:
    print("Unable to connect with DB. Make sure properly given")
    sys.exit(1)

connected


Creating the Brozen Table with Raw data

In [3]:
# Drop the table if it exists
session.execute(
    """ 
    DROP TABLE IF EXISTS bigdata_1.sales_brozen;
    """
)

# Create the table
session.execute(
    """ 
    CREATE TABLE bigdata_1.sales_brozen (
        sales_id int PRIMARY KEY,
        Region text,
        Country text,
        Item_Type text,
        Sales_Channel text,
        Order_Priority text,
        Order_Date text,
        Order_ID int,
        Ship_Date text,
        UnitsSold int,
        UnitPrice float,
        UnitCost float,
        TotalRevenue float,
        TotalCost float,
        TotalProfit float
    );
    """
)


Reading the CSV Data from GITHUB

In [4]:
import pandas as pd
url = "https://raw.githubusercontent.com/gchandra10/filestorage/refs/heads/main/sales_100.csv"
df = pd.read_csv(url)

Inserting the records into Brozen Table

In [5]:
insert_rows = session.prepare(
        """
        INSERT INTO bigdata_1.sales_brozen (
                                                sales_id,
                                                Region,
                                                Country,
                                                Item_Type,
                                                Sales_Channel,
                                                Order_Priority,
                                                Order_Date,
                                                Order_ID,
                                                Ship_Date,
                                                UnitsSold,
                                                UnitPrice,
                                                UnitCost,
                                                TotalRevenue,
                                                TotalCost,
                                                TotalProfit
                                         ) VALUES (
                                         ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
                                         )
        """,
)

row_id = 1

for index, row in df.iterrows():
    session.execute(insert_rows,(row_id,row["Region"],row["Country"],row['Item Type'],row['Sales Channel'],row['Order Priority'],row['Order Date'],\
                                 row['Order ID'],row['Ship Date'],row['UnitsSold'],row['UnitPrice'],row['UnitCost'],row['TotalRevenue'],row['TotalCost'],row['TotalProfit']))
    row_id+=1

Displaying the Brozen Data

In [6]:
# Fetch all rows from the table
query = "SELECT * FROM bigdata_1.sales_brozen"
rows = session.execute(query)

# Convert the result to a Pandas DataFrame
data = []
for row in rows:
    data.append(row._asdict())  # Convert the row object to a dictionary

df_result = pd.DataFrame(data)

# Display the DataFrame
print(df_result)

    sales_id               country        item_type  order_date   order_id  \
0         23          Turkmenistan       Vegetables   4/13/2015  116205585   
1         53  Antigua and Barbuda   Office Supplies   10/2/2011  286891067   
2         91              Pakistan             Meat  12/28/2013  500371730   
3         55              Thailand             Meat    2/4/2015  252889239   
4         33                 India           Snacks  10/10/2012  440306556   
..       ...                   ...              ...         ...        ...   
94        70               Finland       Vegetables   10/9/2012  642134416   
95        12               Algeria        Cosmetics   2/18/2011  761723172   
96        90               Romania           Cereal   4/16/2015  633134210   
97        77              Thailand           Fruits   9/30/2012  434008300   
98         3      Papua New Guinea             Meat   5/15/2015  940995585   

   order_priority                             region sales_chan

Data Cleaning Activities

1. Checking for Duplicates rows

In [7]:
print(df.duplicated().sum())

0


2. Checking for Null Values

In [8]:
print(df.isnull().sum())

Region            0
Country           0
Item Type         0
Sales Channel     0
Order Priority    0
Order Date        0
Order ID          0
Ship Date         0
UnitsSold         0
UnitPrice         0
UnitCost          0
TotalRevenue      0
TotalCost         0
TotalProfit       0
dtype: int64


3. Droping row and Columns with Null Values

In [9]:
df = df.dropna()  # Drop rows with any missing values
df = df.dropna(axis=1)  # Drop columns with missing values


4. Datatype conversion

In [10]:
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%m/%d/%Y', errors='coerce')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], format='%m/%d/%Y', errors='coerce')

Creating the Silver Table

In [11]:
# Drop the table if it exists
session.execute(
    """ 
    DROP TABLE IF EXISTS bigdata_1.sales_silver;
    """
)

# Create the table
session.execute(
    """ 
    CREATE TABLE bigdata_1.sales_silver (
        sales_id int PRIMARY KEY,
        Region text,
        Country text,
        Item_Type text,
        Sales_Channel text,
        Order_Priority text,
        Order_Date date,
        Order_ID int,
        Ship_Date date,
        UnitsSold int,
        UnitPrice float,
        UnitCost float,
        TotalRevenue float,
        TotalCost float,
        TotalProfit float
    );
    """
)


Inserting the records into Silver Table

In [12]:
insert_rows = session.prepare(
        """
        INSERT INTO bigdata_1.sales_silver (
                                                sales_id,
                                                Region,
                                                Country,
                                                Item_Type,
                                                Sales_Channel,
                                                Order_Priority,
                                                Order_Date,
                                                Order_ID,
                                                Ship_Date,
                                                UnitsSold,
                                                UnitPrice,
                                                UnitCost,
                                                TotalRevenue,
                                                TotalCost,
                                                TotalProfit
                                         ) VALUES (
                                         ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
                                         )
        """,
)

row_id = 1

for index, row in df.iterrows():
    session.execute(insert_rows,(row_id,row["Region"],row["Country"],row['Item Type'],row['Sales Channel'],row['Order Priority'],row['Order Date'],\
                                 row['Order ID'],row['Ship Date'],row['UnitsSold'],row['UnitPrice'],row['UnitCost'],row['TotalRevenue'],row['TotalCost'],row['TotalProfit']))
    row_id+=1

Displaying Silver Tables Data 

In [13]:
# Fetch all rows from the table
query = "SELECT * FROM bigdata_1.sales_silver"
rows = session.execute(query)

# Convert the result to a Pandas DataFrame
data = []
for row in rows:
    data.append(row._asdict())  # Convert the row object to a dictionary

df_result = pd.DataFrame(data)

# Display the DataFrame
print(df_result)

    sales_id               country        item_type  order_date   order_id  \
0         23          Turkmenistan       Vegetables  2015-04-13  116205585   
1         53  Antigua and Barbuda   Office Supplies  2011-10-02  286891067   
2         91              Pakistan             Meat  2013-12-28  500371730   
3         55              Thailand             Meat  2015-02-04  252889239   
4         33                 India           Snacks  2012-10-10  440306556   
..       ...                   ...              ...         ...        ...   
94        70               Finland       Vegetables  2012-10-09  642134416   
95        12               Algeria        Cosmetics  2011-02-18  761723172   
96        90               Romania           Cereal  2015-04-16  633134210   
97        77              Thailand           Fruits  2012-09-30  434008300   
98         3      Papua New Guinea             Meat  2015-05-15  940995585   

   order_priority                             region sales_chan

Aggregating the Data by Region, Country, Item Type and Order Date

In [14]:
df['Shipping_Duration'] = (df['Ship Date'] - df['Order Date']).dt.days
df['Profit_Margin'] = (df['TotalProfit'] / df['TotalRevenue']) * 100
gold_table = df.groupby(['Region', 'Country', 'Item Type', 'Order Date'], as_index=False).agg(
    Total_UnitsSold=('UnitsSold', 'sum'),
    Total_Revenue=('TotalRevenue', 'sum'),
    Total_Profit=('TotalProfit', 'sum'),
    Average_Profit_Margin=('Profit_Margin', 'mean'),
    Average_Shipping_Duration=('Shipping_Duration', 'mean')
)

Creating the Gold Table

In [20]:
# Calculate Total Revenue, Cost, and Profit by Region and Country
gold_table_1 = df.groupby(['Region', 'Country']).agg({
    'TotalRevenue': 'sum',
    'TotalCost': 'sum',
    'TotalProfit': 'sum'
}).reset_index()

In [21]:
gold_table_1

,Region,Country,TotalRevenue,TotalCost,TotalProfit
0,Asia,Brunei,868465.40,494438.42,374026.98
1,Asia,China,3771157.11,3040043.36,731113.75
2,Asia,India,816150.42,521206.56,294943.86
3,Asia,Indonesia,1176911.59,840453.44,336458.15
4,Asia,Japan,5608790.11,4217818.22,1390971.89
...,...,...,...,...,...
75,Sub-Saharan Africa,Tanzania,3555764.47,2158894.56,1396869.91
76,Sub-Saharan Africa,The Gambia,81161.67,60197.08,20964.59
77,Sub-Saharan Africa,Togo,2101183.20,1265563.98,835619.22
78,Sub-Saharan Africa,Uganda,2673613.43,1613701.40,1059912.03


In [27]:
# Calculate Sales Performance by Item Type
gold_table_2 = df.groupby('Item Type').agg({
    'TotalRevenue': 'sum',
    'TotalCost': 'sum',
    'TotalProfit': 'sum'
}).reset_index()

print(gold_table_2)


          Item Type  TotalRevenue    TotalCost  TotalProfit
0         Baby Food    5200564.16   3247704.24   1952859.92
1         Beverages    2145024.70   1437098.74    707925.96
2            Cereal    9416123.20   5360827.36   4055295.84
3           Clothes    4387373.44   1438904.32   2948469.12
4         Cosmetics   28727100.40  17302624.31  11424476.09
5            Fruits     615033.60    456166.40    158867.20
6         Household   38519082.80  28966405.60   9552677.20
7              Meat   21278865.93  18393869.53   2884996.40
8   Office Supplies   27880904.94  22475637.44   5405267.50
9     Personal Care    3191147.85   2212680.15    978467.70
10           Snacks    2193642.66   1400894.88    792747.78
11       Vegetables    1135114.08    669972.24    465141.84


In [30]:
# Calculate Units Sold and Average Unit Price by Order Priority and Sales Channel
gold_table_3 = df.groupby(['Order Priority', 'Sales Channel']).agg({
    'UnitsSold': 'sum',
    'UnitPrice': 'mean'
}).reset_index()

print(gold_table_3)


  Order Priority Sales Channel  UnitsSold   UnitPrice
0              C       Offline      38003  457.407500
1              C        Online      54168  271.538182
2              H       Offline      50026  312.753000
3              H        Online      71680  261.349167
4              L       Offline      59931  154.047000
5              L        Online      59696  230.461429
6              M       Offline      38530  216.261667
7              M        Online     122776  298.861364


In [38]:
# Drop the table if it exists
session.execute(
    """ 
    DROP TABLE IF EXISTS bigdata_1.gold_sales_rev_country;
    """
)

session.execute(
    """ 
    CREATE TABLE bigdata_1.gold_sales_rev_country (
    Rev_ID int primary key,
    Region TEXT,
    Country TEXT,
    TotalRevenue FLOAT,
    TotalCost FLOAT,
    TotalProfit FLOAT
);

    """
)


Inserting Records into Gold Table

In [39]:
gold_table_1.columns

Index(['Region', 'Country', 'TotalRevenue', 'TotalCost', 'TotalProfit'], dtype='object')

In [40]:

insert_query = session.prepare("""
INSERT INTO bigdata_1.gold_sales_rev_country (
    Rev_ID,
    Region,
    Country,
    TotalRevenue,
    TotalCost,
    TotalProfit
) VALUES (?, ?, ?, ?, ?, ?);
"""
)
count=1
# Insert data from a Pandas DataFrame
for _, row in gold_table_1.iterrows():
    session.execute(insert_query, (
        count,
        row['Region'],
        row['Country'], 
        row['TotalRevenue'],
        row['TotalCost'],
        row['TotalProfit']
    ))
    count+=1


Displaying Few Aggregation results

In [41]:
# Execute the query
rows = session.execute(
    """
    SELECT * FROM bigdata_1.gold_sales_rev_country limit 5;
    """
)

# Print the rows
for row in rows:
    print(row)


Row(rev_id=23, country='Antigua and Barbuda ', region='Central America and the Caribbean', totalcost=3305673.0, totalprofit=794996.25, totalrevenue=4100669.25)
Row(rev_id=53, country='Kuwait', region='Middle East and North Africa', totalcost=736723.625, totalprofit=242960.1875, totalrevenue=979683.8125)
Row(rev_id=55, country='Morocco', region='Middle East and North Africa', totalcost=165258.234375, totalprofit=338631.84375, totalrevenue=503890.09375)
Row(rev_id=33, country='Czech Republic', region='Europe', totalcost=2411312.75, totalprofit=1592127.625, totalrevenue=4003440.5)
Row(rev_id=5, country='Japan', region='Asia', totalcost=4217818.0, totalprofit=1390971.875, totalrevenue=5608790.0)


In [42]:
gold_table_2.columns

Index(['Item Type', 'TotalRevenue', 'TotalCost', 'TotalProfit'], dtype='object')

In [43]:
# Drop the table if it exists
session.execute(
    """ 
    DROP TABLE IF EXISTS bigdata_1.gold_sales_performance_by_item_type;
    """
)


session.execute(
    """ 
    CREATE TABLE bigdata_1.gold_sales_performance_by_item_type (
    Sales_ID int primary key,
    Item_type TEXT,
    TotalRevenue FLOAT,
    TotalCost FLOAT,
    TotalProfit FLOAT
);

    """
)


In [44]:

insert_query = session.prepare("""
INSERT INTO bigdata_1.gold_sales_performance_by_item_type (
    Sales_ID,
    Item_type,
    TotalRevenue,
    TotalCost,
    TotalProfit
) VALUES (?, ?, ?, ?, ?);
"""
)
count=1
# Insert data from a Pandas DataFrame
for _, row in gold_table_2.iterrows():
    session.execute(insert_query, (
        count,
        row['Item Type'], 
        row['TotalRevenue'],
        row['TotalCost'],
        row['TotalProfit']
    ))
    count+=1


In [46]:
rows=session.execute(
    """ 
    SELECT * FROM bigdata_1.gold_sales_performance_by_item_type LIMIT 5;
    """
)


# Print the rows
for row in rows:
    print(row)

Row(sales_id=5, item_type='Cosmetics', totalcost=17302624.0, totalprofit=11424476.0, totalrevenue=28727100.0)
Row(sales_id=10, item_type='Personal Care', totalcost=2212680.25, totalprofit=978467.6875, totalrevenue=3191147.75)
Row(sales_id=11, item_type='Snacks', totalcost=1400894.875, totalprofit=792747.75, totalrevenue=2193642.75)
Row(sales_id=1, item_type='Baby Food', totalcost=3247704.25, totalprofit=1952859.875, totalrevenue=5200564.0)
Row(sales_id=8, item_type='Meat', totalcost=18393870.0, totalprofit=2884996.5, totalrevenue=21278866.0)


In [56]:
# Drop the table if it exists
session.execute(
    """ 
    DROP TABLE IF EXISTS bigdata_1.gold_order_priority_by_sales;
    """
)


session.execute(
    """ 
    CREATE TABLE bigdata_1.gold_order_priority_by_sales (
    Order_priority_ID int primary key,
    Order_priority TEXT,
    Sales_channel TEXT,
    UnitPrice FLOAT,
    UnitsSold INT
);

    """
)


In [58]:

insert_query = session.prepare("""
INSERT INTO bigdata_1.gold_order_priority_by_sales (
    Order_priority_ID,
    Order_priority,
    Sales_channel,
    UnitPrice,
    UnitsSold
) VALUES (?, ?, ?, ?,?);
"""
)
count=1
# Insert data from a Pandas DataFrame
for _, row in gold_table_3.iterrows():
    session.execute(insert_query, (
        count,
        row['Order Priority'], 
        row['Sales Channel'],
        row['UnitPrice'],
        row['UnitsSold']
    ))
    count+=1


In [59]:
rows=session.execute(
    """ 
    SELECT * FROM bigdata_1.gold_order_priority_by_sales LIMIT 5;
    """
)


# Print the rows
for row in rows:
    print(row)

Row(order_priority_id=5, order_priority='L', sales_channel='Offline', unitprice=154.0469970703125, unitssold=59931)
Row(order_priority_id=1, order_priority='C', sales_channel='Offline', unitprice=457.4075012207031, unitssold=38003)
Row(order_priority_id=8, order_priority='M', sales_channel='Online', unitprice=298.8613586425781, unitssold=122776)
Row(order_priority_id=2, order_priority='C', sales_channel='Online', unitprice=271.5381774902344, unitssold=54168)
Row(order_priority_id=4, order_priority='H', sales_channel='Online', unitprice=261.3491516113281, unitssold=71680)


In [62]:
query = "SELECT table_name FROM system_schema.tables WHERE keyspace_name = 'bigdata_1';"
result = session.execute(query)
# Print the tables
for table in result:
    print(table)

Row(table_name='developers_by_language')
Row(table_name='gold_order_priority_by_sales')
Row(table_name='gold_sales_performance_by_item_type')
Row(table_name='gold_sales_rev_country')
Row(table_name='monster_by_city')
Row(table_name='monster_by_family')
Row(table_name='sales_brozen')
Row(table_name='sales_silver')
Row(table_name='test')
Row(table_name='users')
